In [ ]:
import pandas as pd

file_path = "../data/01_raw/PlantCLEF2022_trusted_training_metadata.csv"
try:
    df = pd.read_csv(file_path, delimiter=";")
    if df.empty:
        print("The CSV file is empty")
    else:
        print(df.head(1))

except FileNotFoundError:
    print(
        f"File not found: {file_path}. Please check the file path and that the file is downloaded."
    )

In [ ]:
# Drop columns that are not needed
df = df.drop(
    columns=[
        "image_name",
        "source",
        "manual_tag",
        "predicted_tag",
        "predicted_tag_probability",
        "original_url",
        "license",
        "publisher",
        "gbif_occurrence_id",
        "aggregator",
        "dataset_key",
    ]
)

In [ ]:
# amount of rows
print(df.shape[0])

# amount of unique species
print(df["species"].nunique())

# print unique classes
print(df["species"].unique())

This are way too many species to train on any device in a reasonable time. Some species might not be necessary to train on, because they are not present in switzerland. I will try to find out which species are present in switzerland and only train on those.

lets load the list with invasive species from https://www.infoflora.ch/de/neophyten/listen-und-infoblätter.html and see which species are present in switzerland.

In [ ]:
file_path = "../data/01_raw/liste-inv-neoph-ch-2021-d-f-i.csv"
try:
    df_neophytes = pd.read_csv(file_path, delimiter=";")
    if df_neophytes.empty:
        print("The CSV file is empty")
    else:
        print(df_neophytes.head(1))

except FileNotFoundError:
    print(
        f"File not found: {file_path}. Please check the file path and that the file is downloaded."
    )

minimize the df that only species appear that are present in switzerland and classified.

In [ ]:
# Preprocessing
def normalize_species(name):
    return name.lower().strip()

df['species'] = df['species'].apply(normalize_species)
df_neophytes['species'] = df_neophytes['Wissenschaftlicher Name'].apply(normalize_species)

# Merge dataframes on 'species'
merged_df = df.merge(df_neophytes[['species']], on='species', how='inner')

# Select desired columns
result = merged_df[df.columns]

print(result)

In [ ]:
# save the df to a csv file
result.to_csv("../data/02_processed/merged_data.csv", index=False, sep=";")